based off of Mornan's model

In [26]:
# IMPORTS!

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 200)

In [27]:
air_walk = pd.read_csv('./data/air_walk_merged_clean.csv')
air_walk.head(2)

,name,pm_2.5,temp_f,model,hidden,flagged,downgraded,age,is_owner,1week_avg,fips_id,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,CBSA_Name,CBSA_POP,CBSA_EMP,CBSA_WRK,Ac_Total,Ac_Water,Ac_Land,Ac_Unpr,TotPop,CountHU,HH,P_WrkAge,Pct_AO0,Pct_AO1,Pct_AO2p,Workers,R_LowWageWk,R_MedWageWk,R_HiWageWk,R_PCTLOWWAGE,TotEmp,E5_Ret,E5_Off,E5_Ind,E5_Svc,E5_Ent,E8_Ret,E8_off,E8_Ind,E8_Svc,E8_Ent,E8_Ed,E8_Hlth,E8_Pub,E_LowWageWk,E_MedWageWk,E_HiWageWk,E_PctLowWage,D1A,D1B,D1C,D1C5_RET,D1C5_OFF,D1C5_IND,D1C5_SVC,D1C5_ENT,D1C8_RET,D1C8_OFF,D1C8_IND,D1C8_SVC,D1C8_ENT,D1C8_ED,D1C8_HLTH,D1C8_PUB,D1D,D2A_JPHH,D2B_E5MIX,D2B_E5MIXA,D2B_E8MIX,D2B_E8MIXA,D2A_EPHHM,D2C_TRPMX1,D2C_TRPMX2,D2C_TRIPEQ,D2R_JOBPOP,D2R_WRKEMP,D2A_WRKEMP,D2C_WREMLX,D3A,D3AAO,D3AMM,D3APO,D3B,D3BAO,D3BMM3,D3BMM4,D3BPO3,D3BPO4,D4A,D4B025,D4B050,D4C,D4D,D4E,D5AR,D5AE,D5BR,D5BE,D5CR,D5CRI,D5CE,D5CEI,D5DR,D5DRI,D5DE,D5DEI,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area,Low_transit_flag_dist,Low_transit_flag_freq,Pct_Water,Pct_Land,Pct_Unpr,Log_Acres,log_TotPop,Pct_HH_occupied,log_D2A_JPHH,log_D2A_WRKEMP,Pct_Workers,pct_E5_Ret,pct_E5_Off,pct_E5_Ind,pct_E5_Svc,pct_E5_Ent,pct_E8_Ret,pct_E8_off,pct_E8_Ind,pct_E8_Svc,pct_E8_Ent,pct_E8_Ed,pct_E8_Hlth,pct_E8_Pub,log_D3AAO,log_D3AMM,log_D3APO,log_D3B,log_D3BAO,log_D3BMM3,log_D3BMM4,log_D3BPO3,log_D3BPO4,log_D4A,log_D4C,log_D4D,log_D4E,log_D5AE,log_D5BR,log_D5BE,log_D5CR,pow_D5CRI,pow_D5CE,pow_D5CEI,pow_D5DR,pow_D5DRI,pow_D5DE,pow_D5DEI,pow_D2A_Ranked,log_1week_avg
0,#ValleyClimate,19.54,84.0,PMS5003+PMS5003+BME280,False,False,False,1,False,43.48,60190050003,6,19,5000,3,260.0,"Fresno-Madera-Hanford, CA",23420.0,"Fresno, CA",978130,370040,371133,318.801501,3.41937,315.382131,315.382131,1653,730.0,663.0,0.615,0.075415,0.398190,0.526395,597,155,239,203,0.259631,972,152,59,185,362,214,152,59,185,326,214,0,36,0,383,395,194,0.394033,2.314652,5.241261,3.081975,0.481955,0.187075,0.586590,1.147814,0.678542,0.481955,0.187075,0.586590,1.033667,0.678542,0.000000,0.114147,0.0,5.396628,1.466063,0.917728,0.917728,0.891662,0.768304,0.866894,0.772433,0.830073,0.496555,0.740571,0.760994,0.614198,0.679905,20.733482,0.000000,5.260424,15.473059,89.976435,0.00000,4.058569,24.351411,30.439264,42.614970,513.65,0.0,0.0,11.00,22.322127,0.006655,81753,73789,85266,54825,0.002845,0.922325,0.002718,0.943919,0.003773,0.586928,0.003201,0.500064,20,18,13,15,15.666667,4825.986205,1.290175e+06,False,False,0.010726,0.989274,0.989274,5.767700,7.410952,0.908219,0.902623,0.478838,0.361162,0.156379,0.0607,0.190329,0.372428,0.220165,0.156379,0.0607,0.190329,0.335391,0.220165,0.000000,0.037037,0.0,3.078854,0.000000,1.834248,2.801726,4.510601,0.00000,1.621084,3.232834,3.448058,3.775400,2.484907,3.149403,0.006633,11.208979,11.353543,10.911920,0.002841,0.991947,0.553883,0.994245,0.572362,0.948110,0.563022,0.933045,1.349283,3.795040
1,'S' St Between Inyo and Mono,18.52,93.0,PMS5003+PMS5003+BME280,False,False,False,1,False,41.81,60190005021,6,19,502,1,260.0,"Fresno-Madera-Hanford, CA",23420.0,"Fresno, CA",978130,370040,371133,110.336080,0.00000,110.336080,110.336080,953,408.0,381.0,0.599,0.081365,0.417323,0.501312,263,97,109,57,0.368821,8559,58,0,47,7913,541,58,0,47,157,541,7,7749,0,851,2495,5213,0.099428,3.697793,8.637247,77.572087,0.525667,0.000000,0.425971,71.717248,4.903201,0.525667,0.000000,0.425971,1.422925,4.903201,0.063443,70.230880,0.0,81.269880,22.464567,0.223268,0.192313,0.226667,0.195308,0.293581,0.223464,0.246951,0.373636,0.200378,0.059624,0.030728,0.379359,25.092344,4.460549,6.869627,13.762167,102.505726,11.60092,34.802759,23.201839,58.004598,17.401379,0.00,0.0,0.0,15.33,88.921049,0.016086,81602,71873,93553,65891,0.002840,0.920621,0.002647,0.919410,0.004140,0.643972,0.003847,0.600998,5,2,14,20,12.500000,3072.308296,4.465223e+05,False,False,0.000000,1.000000,1.000000,4.712553,6.860664,0.933824,3.155491,0.030265,0.275971,0.006776,0.0000,0.005491,0.924524,0.063208,0.006776,0.0000,0.005491,0.018343,0.063208,0.000818,0.905363,0.0,3.261642,1.6

In [28]:
# Baseline model

from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_dum = air_walk[['NatWalkInd']]
y_dum = air_walk['1week_avg']

X_train, X_test, y_train, y_test = train_test_split(X_dum, y_dum, random_state=42)

dummy = DummyRegressor(strategy="mean")
dummy.fit(X_train, y_train)
preds_dum = dummy.predict(X_test)
print(dummy.score(X_test, y_test))
print(mean_squared_error(y_test, preds_dum, squared=False))

-0.000520466217100557
7.588526393381297


In [41]:
# More imports for modeling

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE, SequentialFeatureSelector
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV

In [30]:
X = air_walk.select_dtypes(exclude=object).drop(
    columns = ['log_1week_avg', '1week_avg', 'pm_2.5'])
X.isna().sum()[X.isna().sum() > 0]

CSA     986
CBSA    242
dtype: int64

In [31]:
X.drop(columns = ['CSA', 'CBSA'], inplace = True)

In [32]:
(X == float('inf')).sum()[(X == float('inf')).sum() > 0]

Series([], dtype: int64)

In [37]:
X.head()

,temp_f,hidden,flagged,downgraded,age,is_owner,fips_id,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CBSA_POP,CBSA_EMP,CBSA_WRK,Ac_Total,Ac_Water,Ac_Land,Ac_Unpr,TotPop,CountHU,HH,P_WrkAge,Pct_AO0,Pct_AO1,Pct_AO2p,Workers,R_LowWageWk,R_MedWageWk,R_HiWageWk,R_PCTLOWWAGE,TotEmp,E5_Ret,E5_Off,E5_Ind,E5_Svc,E5_Ent,E8_Ret,E8_off,E8_Ind,E8_Svc,E8_Ent,E8_Ed,E8_Hlth,E8_Pub,E_LowWageWk,E_MedWageWk,E_HiWageWk,E_PctLowWage,D1A,D1B,D1C,D1C5_RET,D1C5_OFF,D1C5_IND,D1C5_SVC,D1C5_ENT,D1C8_RET,D1C8_OFF,D1C8_IND,D1C8_SVC,D1C8_ENT,D1C8_ED,D1C8_HLTH,D1C8_PUB,D1D,D2A_JPHH,D2B_E5MIX,D2B_E5MIXA,D2B_E8MIX,D2B_E8MIXA,D2A_EPHHM,D2C_TRPMX1,D2C_TRPMX2,D2C_TRIPEQ,D2R_JOBPOP,D2R_WRKEMP,D2A_WRKEMP,D2C_WREMLX,D3A,D3AAO,D3AMM,D3APO,D3B,D3BAO,D3BMM3,D3BMM4,D3BPO3,D3BPO4,D4A,D4B025,D4B050,D4C,D4D,D4E,D5AR,D5AE,D5BR,D5BE,D5CR,D5CRI,D5CE,D5CEI,D5DR,D5DRI,D5DE,D5DEI,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area,Low_transit_flag_dist,Low_transit_flag_freq,Pct_Water,Pct_Land,Pct_Unpr,Log_Acres,log_TotPop,Pct_HH_occupied,log_D2A_JPHH,log_D2A_WRKEMP,Pct_Workers,pct_E5_Ret,pct_E5_Off,pct_E5_Ind,pct_E5_Svc,pct_E5_Ent,pct_E8_Ret,pct_E8_off,pct_E8_Ind,pct_E8_Svc,pct_E8_Ent,pct_E8_Ed,pct_E8_Hlth,pct_E8_Pub,log_D3AAO,log_D3AMM,log_D3APO,log_D3B,log_D3BAO,log_D3BMM3,log_D3BMM4,log_D3BPO3,log_D3BPO4,log_D4A,log_D4C,log_D4D,log_D4E,log_D5AE,log_D5BR,log_D5BE,log_D5CR,pow_D5CRI,pow_D5CE,pow_D5CEI,pow_D5DR,pow_D5DRI,pow_D5DE,pow_D5DEI,pow_D2A_Ranked
0,84.0,False,False,False,1,False,60190050003,6,19,5000,3,978130,370040,371133,318.801501,3.419370,315.382131,315.382131,1653,730.0,663.0,0.615,0.075415,0.398190,0.526395,597,155,239,203,0.259631,972,152,59,185,362,214,152,59,185,326,214,0,36,0,383,395,194,0.394033,2.314652,5.241261,3.081975,0.481955,0.187075,0.586590,1.147814,0.678542,0.481955,0.187075,0.586590,1.033667,0.678542,0.000000,0.114147,0.000000,5.396628,1.466063,0.917728,0.917728,0.891662,0.768304,0.866894,0.772433,0.830073,0.496555,0.740571,0.760994,0.614198,0.679905,20.733482,0.000000,5.260424,15.473059,89.976435,0.00000,4.058569,24.351411,30.439264,42.614970,513.65,0.0,0.0,11.00,22.322127,0.006655,81753,73789,85266,54825,0.002845,0.922325,0.002718,0.943919,0.003773,0.586928,0.003201,0.500064,20,18,13,15,15.666667,4825.986205,1.290175e+06,False,False,0.010726,0.989274,0.989274,5.767700,7.410952,0.908219,0.902623,0.478838,0.361162,0.156379,0.060700,0.190329,0.372428,0.220165,0.156379,0.060700,0.190329,0.335391,0.220165,0.000000,0.037037,0.000000,3.078854,0.000000,1.834248,2.801726,4.510601,0.00000,1.621084,3.232834,3.448058,3.775400,2.484907,3.149403,0.006633,11.208979,11.353543,10.911920,0.002841,0.991947,0.553883,0.994245,0.572362,0.948110,0.563022,0.933045,1.349283
1,93.0,False,False,False,1,False,60190005021,6,19,502,1,978130,370040,371133,110.336080,0.000000,110.336080,110.336080,953,408.0,381.0,0.599,0.081365,0.417323,0.501312,263,97,109,57,0.368821,8559,58,0,47,7913,541,58,0,47,157,541,7,7749,0,851,2495,5213,0.099428,3.697793,8.637247,77.572087,0.525667,0.000000,0.425971,71.717248,4.903201,0.525667,0.000000,0.425971,1.422925,4.903201,0.063443,70.230880,0.000000,81.269880,22.464567,0.223268,0.192313,0.226667,0.195308,0.293581,0.223464,0.246951,0.373636,0.200378,0.059624,0.030728,0.379359,25.092344,4.460549,6.869627,13.762167,102.505726,11.60092,34.802759,23.201839,58.004598,17.401379,0.00,0.0,0.0,15.33,88.921049,0.016086,81602,71873,93553,65891,0.002840,0.920621,0.002647,0.919410,0.004140,0.643972,0.003847,0.600998,5,2,14,20,12.500000,3072.308296,4.465223e+05,False,False,0.000000,1.000000,1.000000,4.712553,6.860664,0.933824,3.155491,0.030265,0.275971,0.006776,0.000000,0.005491,0.924524,0.063208,0.006776,0.000000,0.005491,0.018343,0.063208,0.000818,0.905363,0.000000,3.261642,1.697549,2.063011,2.692068,4.639627,2.53377,3.578025,3.186429,4.077615,2.912426,2.793004,4.498932,0.015958,11.182670,11.446294,11.095772,0.002836,0.991763,0.552428,0.991633,0.577695,0.956944,0.573469,0.950358,1.174619
2,81.0,False,False,False,1,False,60816089003,6,81,608900,3

In [33]:
y = air_walk['log_1week_avg']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

pipe1 = make_pipeline(
    StandardScaler(), 
    RFE(estimator=LinearRegression(), n_features_to_select=10),
    LinearRegression()
)

pipe1.fit(X_train, y_train)
preds1 = pipe1.predict(X_test)
print(pipe1.score(X_train, y_train))
print(pipe1.score(X_test, y_test))
print(mean_squared_error(y_test, preds1, squared=False))

0.02063355502373687
0.00790292818866778
0.7066116241707662


In [36]:
pipe1.named_steps['rfe'].get_feature_names_out()

array(['x30', 'x32', 'x33', 'x34', 'x35', 'x37', 'x43', 'x125', 'x130',
       'x136'], dtype=object)

In [40]:
pipe1.named_steps['standardscaler'].get_feature_names_out()

array(['temp_f', 'hidden', 'flagged', 'downgraded', 'age', 'is_owner',
       'fips_id', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE',
       'CBSA_POP', 'CBSA_EMP', 'CBSA_WRK', 'Ac_Total', 'Ac_Water',
       'Ac_Land', 'Ac_Unpr', 'TotPop', 'CountHU', 'HH', 'P_WrkAge',
       'Pct_AO0', 'Pct_AO1', 'Pct_AO2p', 'Workers', 'R_LowWageWk',
       'R_MedWageWk', 'R_HiWageWk', 'R_PCTLOWWAGE', 'TotEmp', 'E5_Ret',
       'E5_Off', 'E5_Ind', 'E5_Svc', 'E5_Ent', 'E8_Ret', 'E8_off',
       'E8_Ind', 'E8_Svc', 'E8_Ent', 'E8_Ed', 'E8_Hlth', 'E8_Pub',
       'E_LowWageWk', 'E_MedWageWk', 'E_HiWageWk', 'E_PctLowWage', 'D1A',
       'D1B', 'D1C', 'D1C5_RET', 'D1C5_OFF', 'D1C5_IND', 'D1C5_SVC',
       'D1C5_ENT', 'D1C8_RET', 'D1C8_OFF', 'D1C8_IND', 'D1C8_SVC',
       'D1C8_ENT', 'D1C8_ED', 'D1C8_HLTH', 'D1C8_PUB', 'D1D', 'D2A_JPHH',
       'D2B_E5MIX', 'D2B_E5MIXA', 'D2B_E8MIX', 'D2B_E8MIXA', 'D2A_EPHHM',
       'D2C_TRPMX1', 'D2C_TRPMX2', 'D2C_TRIPEQ', 'D2R_JOBPOP',
       'D2R_WRKEMP', 'D2A_WRKEMP', '

In [43]:
y = air_walk['log_1week_avg']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

pipe2 = make_pipeline(
    StandardScaler(), 
    SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select=10),
    LinearRegression()
)

pipe2.fit(X_train, y_train)
preds2 = pipe2.predict(X_test)
print(pipe2.score(X_train, y_train))
print(pipe2.score(X_test, y_test))
print(mean_squared_error(y_test, preds2, squared=False))

0.3073856873173968
0.3011380383717246
0.5930610403594091


In [45]:
pipe2.named_steps['sequentialfeatureselector'].get_feature_names_out()

array(['x0', 'x6', 'x11', 'x12', 'x24', 'x88', 'x111', 'x120', 'x150',
       'x154'], dtype=object)